# Model training

This notebook takes numpy arrays prepared from ASTER satellite imagery using the same functions as are found in the module `array_processing`. Not found in that module is the simple function from the `rasterio` package that reads a geoTIFF file and returns a numpy array.

File names and labels of the data were previously generated as python lists and saved using pickle.

5% of the data has been withheld for final testing purposes.

## Imports

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from functools import partial
import numpy as np
import pickle
import os

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

## Read in all the training data

use the command `!cp - r source_folder destination_folder` to copy all training arrays into colab session storage.

In [9]:
file_names_path = r"G:\.shortcut-targets-by-id\1F5TKMAk_9oNKo13HfwksVmLXmT-4Wy2n\WBS_Project\Training_data\model_development_file_names_numpy.pickle" # path to the pickle object containing the list of file names used in model development
with open(file_names_path, 'rb') as file:
    file_names = pickle.load(file)

In [10]:
data_dir = r"G:\.shortcut-targets-by-id\1F5TKMAk_9oNKo13HfwksVmLXmT-4Wy2n\WBS_Project\Training_data\All_Training_Numpys" # local folder containing training arrays

In [11]:
file_paths = [os.path.join(data_dir, file_name) for file_name in file_names if file_name.endswith('.npy')]

In [12]:
labels_path = r"G:\.shortcut-targets-by-id\1F5TKMAk_9oNKo13HfwksVmLXmT-4Wy2n\WBS_Project\Training_data\model_development_labels.pickle" # path to the pickle object containing the list of lables used in model development
with open(labels_path, 'rb') as file:
    labels = pickle.load(file)

## Prepare the tensorflow dataset

The numpy arrays have already been processed, but functions are needed to read them in based on the file names and to set their shape attribute for tensorflow

In [13]:
def load_numpy_file(file_path, label):
    array = np.load(file_path, allow_pickle = True)
    array = np.ndarray.astype(array, np.float32)
    return array, label

def parse_function(file_path, label):
    array, label = tf.numpy_function(load_numpy_file, [file_path, label], [tf.float32, tf.int32])
    return array, label

def fixup_shape_11(images, labels):
    images.set_shape([224, 224, 11])
    labels.set_shape([])
    return images, labels

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

In [15]:
dataset = dataset.shuffle(buffer_size = len(dataset), seed = 42, reshuffle_each_iteration = False)

In [16]:
dataset = dataset.map(parse_function)

In [17]:
dataset = dataset.map(fixup_shape_11)

In [18]:
train_size = round(len(dataset) * 0.8)
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size)

bs = 8
train_ds = train_ds.batch(bs).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(bs).prefetch(tf.data.AUTOTUNE)

## Set up the model

This model is a combination of the pre-trained [EffNetV2-XL(21k)](https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2) with several preceding layers that serve to reduce the original 11 bands to the expected 3 bands.

A DepthPool class is defined to pool the results of convolutions across bands.

In [19]:
class DepthPool(tf.keras.layers.Layer):
    def __init__(self, pool_size = 2, **kwargs):
        super().__init__(**kwargs)
        self.pool_size = pool_size

    def call(self, inputs):
        shape = tf.shape(inputs)
        groups = shape[-1] // self.pool_size
        new_shape = tf.concat([shape[:-1], [groups, self.pool_size]], axis = 0)
        return tf.reduce_max(tf.reshape(inputs, new_shape), axis = -1)

In [20]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size = 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")


In [21]:
model_handle = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2"

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (224, 224, 11)),
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (5, 5), strides = (1, 1), activation = 'relu'),
    DefaultConv2D(filters = 64),
    DepthPool(),
    DefaultConv2D(filters = 16),
    DefaultConv2D(filters = 8),
    DepthPool(),
    DefaultConv2D(filters = 3),

    # hub.KerasLayer(model_handle, trainable=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                          activation = 'sigmoid')
])
model.build((None,)+(224, 224)+(3,))

In [35]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
f1 = tf.keras.metrics.F1Score()

In [36]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', precision, recall, f1])

In [24]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [30]:
import datetime

In [37]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [38]:
CHECKPOINT_DIR = r"P:\Eli\Mining_for_the_Future\porphyry-copper-deposit-prediction\checkpoints" # File path for saving model checkpoints
callbacks = [
    ModelCheckpoint(CHECKPOINT_DIR, monitor='val_accuracy', save_weights_only = True, save_best_only= True, mode = 'max'),
    tensorboard_callback
]

In [39]:
compound_fitted = model.fit(train_ds, validation_data = val_ds, epochs = 2, callbacks = callbacks)

Layer DepthPool has arguments ['pool_size']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config
Epoch 1/2


616/616 [==============================] - 392s 632ms/step - loss: 0.6316 - accuracy: 0.7101 - precision: 0.5714 - recall: 0.0028 - val_loss: 0.6255 - val_accuracy: 0.6976 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/2
616/616 [==============================] - 394s 639ms/step - loss: 0.6038 - accuracy: 0.7111 - precision: 0.8750 - recall: 0.0049 - val_loss: 0.6443 - val_accuracy: 0.6984 - val_precision: 0.5000 - val_recall: 0.0027


In [41]:
evaluation_results = model.evaluate(val_ds, return_dict = True)
evaluation_results

154/154 [==============================] - 207s 299ms/step - loss: 0.6443 - accuracy: 0.6984 - precision: 0.5000 - recall: 0.0027


{'loss': 0.6442956328392029,
 'accuracy': 0.6983739733695984,
 'precision': 0.5,
 'recall': 0.002695417730137706}

In [42]:
evaluation_results['recall']

0.002695417730137706

In [44]:
import pandas as pd

In [57]:
test_results = pd.read_csv(r"P:\Eli\Mining_for_the_Future\porphyry-copper-deposit-prediction\experiment_results.csv")

In [58]:
test_results

,Exp_num,Parent,Notes,Sprint_ID,Dataset,Model,Accuracy,Recall,Precision,F1


In [48]:
exp_num = '1'
parent_exp = 'NA'
notes = 'test'
sprint_id = '1'
dataset_id = 'dummy'
model_id = 'dummy'
accuracy = 0.7
recall = 0.69
precision = 0.68
f1 = 0.35

In [51]:
exp_results = pd.DataFrame({
    "Exp_num": exp_num,
    "Parent": parent_exp,
    "Notes": notes,
    "Sprint_ID": sprint_id,
    "Dataset": dataset_id,
    "Model": model_id,
    "Accuracy": accuracy,
    "Recall": recall,
    "Precision": precision,
    "F1": f1
},
index = [0])

In [59]:
results = pd.concat([results, exp_results], axis = 0)
results

,Exp_num,Parent,Notes,Sprint_ID,Dataset,Model,Accuracy,Recall,Precision,F1
0,1,NA,test,1,dummy,dummy,0.7,0.69,0.68,0.35
0,1,NA,test,1,dummy,dummy,0.7,0.69,0.68,0.35
0,1,NA,test,1,dummy,dummy,0.7,0.69,0.68,0.35


In [60]:
results.to_csv(r"P:\Eli\Mining_for_the_Future\porphyry-copper-deposit-prediction\experiment_results.csv", index = False)


In [28]:
%tensorboard --logdir logs/fit

In [ ]:
model_save_path = "" # Folder in which to save the complete model
model.save(model_save_path)

In [3]:
102 // 3

34